In [3]:
import mysql.connector
from mysql.connector import Error
from datetime import datetime

# Define hotel fields and database name
hotel_fields = ['ID', 'Name', 'Age', 'Address', 'Country', 'Email', 'Phone', 'Checkin date', 'Checkout date',
                'Room number', 'Room rent', 'Restaurant bill', 'Laundry bill', 'Gaming bill', 'Total bill']
hotel_database = 'hotelrecords.csv'

def connect():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Mahak@2005",
            database="hotel"
        )
        if conn.is_connected():
            print("Connected to MySQL database")
            return conn
    except Error as e:
        print(e)

def login():
    while True:
        username = input("Enter your username: ")
        password = input("Enter your password: ")

        try:
            conn = connect()
            cursor = conn.cursor()

            cursor.execute("SELECT * FROM users WHERE username = %s AND password = %s", (username, password))
            user = cursor.fetchone()

            if user:
                print("Login successful!")
                return user
            else:
                print("Invalid username or password. Please try again.")
        except Error as e:
            print(e)
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()
                print("Connection closed")

# Function to add a customer
def add_Customer():
    print("-------------------------")
    print("Add Customer Information")
    print("-------------------------")
    try:
        conn = connect()
        cursor = conn.cursor()

        Customer_data = []
        for i in range(9):
            value = input("Enter " + hotel_fields[i + 1] + ": ")
            Customer_data.append(value)

        print("\n***** Available Room Choices *****")
        print("1. Ultra Royal - 10000 Rs.")
        print("2. Royal - 5000 Rs.")
        print("3. Elite - 3500 Rs.")
        print("4. Budget - 2500 Rs.\n")
        roomchoice = int(input("Enter Room Choice Number: "))
        noofdays = int(input("For How Many Nights Did You Stay: "))
        roomno = int(input("Enter Customer Room No : "))
        Customer_data.append(roomno)

        roomrent = 0
        if roomchoice == 1:
            roomrent = noofdays * 10000
        elif roomchoice == 2:
            roomrent = noofdays * 5000
        elif roomchoice == 3:
            roomrent = noofdays * 3500
        elif roomchoice == 4:
            roomrent = noofdays * 2500
        else:
            print("Invalid choice!")
            return

        Customer_data.append(roomrent)
        Customer_data.extend([0] * 4)  # Extend list to include 0s for additional bill fields

        # Insert customer data into database
        cursor.execute("""INSERT INTO hotel_records (Name, Age, Address, Country, Email, Phone, Checkin_date, 
                                                      Checkout_date, Room_number, Room_rent, Restaurant_bill,
                                                      Laundry_bill, Gaming_bill, Total_bill)
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                       tuple(Customer_data))
        conn.commit()
        print("Data saved successfully")
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")


# Function to calculate restaurant bill
def restaurant_bill():
    try:
        conn = connect()
        cursor = conn.cursor()

        ID = int(input("Enter your ID: "))
        print("\n***** RESTAURANT MENU *****\n")
        print("1. water -----> Rs20")
        print("2. tea -----> Rs10")
        print("3. breakfast combo ---> Rs90")
        print("4. lunch ----> Rs110")
        print("5. dinner ---> Rs150")
        print("6. Exit\n")

        choice = int(input("Enter your choice: "))
        quantity = int(input("Enter the quantity: "))
        menu = {1: 20, 2: 10, 3: 90, 4: 110, 5: 150}
        rtbill = menu.get(choice, 0) * quantity

        # Update restaurant bill in the database
        cursor.execute("UPDATE hotel_records SET Restaurant_bill = Restaurant_bill + %s WHERE ID = %s",
                       (rtbill, ID))
        conn.commit()
        print("Total Food Cost = Rs", rtbill)
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")

# Function to delete a record
def delete():
    try:
        conn = connect()
        cursor = conn.cursor()

        ID = input("Enter Customer ID you want to delete: ")
        cursor.execute("DELETE FROM hotel_records WHERE ID = %s", (ID,))
        conn.commit()
        print("Record deleted Successfully")
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")

# Function to show all customer records
def show():
    try:
        conn = connect()
        cursor = conn.cursor()

        cursor.execute("SELECT * FROM hotel_records")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")

# Function to search customer by ID
def search():
    try:
        conn = connect()
        cursor = conn.cursor()

        ID = input('Enter Customer ID You Want To Search: ')
        cursor.execute("SELECT * FROM hotel_records WHERE ID = %s", (ID,))
        row = cursor.fetchone()
        if row:
            print(row)
        else:
            print('No Customer Data Found!')
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")

# Function to generate total bill amount
def totalbill():
    try:
        conn = connect()
        cursor = conn.cursor()

        ID = input('Enter Customer ID : ')
        cursor.execute("SELECT * FROM hotel_records WHERE ID = %s", (ID,))
        row = cursor.fetchone()
        if row:
            print("\n****** HOTEL BILL ******")
            print("Customer ID:", row[0])
            print("Customer Name:", row[1])
            total_bill = row[-1]  # Assuming the last element is the Total bill in the row
            print("Total Bill:", total_bill)
        else:
            print("Customer not found")
    except Error as e:
        print(e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("Connection closed")

def main():
    print("-------------------------------")
    print("            WELCOME            ")
    print("-------------------------------")
    # Authenticate user
    user = login()
    if user:
        print("Welcome,", user[1])  # Assuming user[1] is the username column in the database
        while True:
            print("""
            1. Enter Customer Details 
            2. Calculate Restaurant Bill 
            3. Delete Customer Details
            4. Show All Customer Records
            5. Search Customer
            6. Generate Total Bill Amount
            7. Exit
            """)
            choice = input("Enter Your Choice : ")
            if choice == '1':
                add_Customer()
            elif choice == '2':
                restaurant_bill()
            elif choice == '3':
                delete()
            elif choice == '4':
                show()
            elif choice == '5':
                search()
            elif choice == '6':
                totalbill()
            elif choice == '7':
                print("-------------------------------")
                print(" Thank you for using our system")
                print("-------------------------------")
                break
            else:
                print("Invalid Choice")
            continue_or_exit = input("Press 1 to return to the menu or 0 to exit: ")
            if continue_or_exit == '0':
                break
    else:
        print("Authentication failed. Exiting...")

if __name__ == "__main__":
    main()

           


-------------------------------
            WELCOME            
-------------------------------
Enter your username: Manager
Enter your password: password
Connected to MySQL database
Login successful!
Connection closed
Welcome, Manager

            1. Enter Customer Details 
            2. Calculate Restaurant Bill 
            3. Delete Customer Details
            4. Show All Customer Records
            5. Search Customer
            6. Generate Total Bill Amount
            7. Exit
            
Enter Your Choice : 7
-------------------------------
 Thank you for using our system
-------------------------------
